데이터 분석용 전처리.
Word2Vec을 활용한 문자열 데이터 임베딩 전처리.

In [1]:
# Data Analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
plt.style.use('seaborn')
sns.set_style("whitegrid")

# Modelling
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from scipy import stats

# Additional
import os
import math
import random
import itertools
import multiprocessing
from tqdm import tqdm
from time import time
import logging
import pickle

작업 경로 설정

In [2]:
os.chdir(r'C:\Users\styli\Desktop\github_repository_add\Github_cowork_practice')

다른 데이터의 전처리는 간단하게 요약이 되어져 있다.
분석 직전단계에, scaling만 사용하면 될것이지만, 문자열 단어의 경우에는 매핑이 필요할것으로 예상된다.

train, test셋에 있는 모든 데이터를 활용해 메뉴 데이터 얻기.

In [3]:
data1 = pd.read_csv('train.csv')
data2 = pd.read_csv('test.csv')
full_data = pd.concat([data1, data2],axis=0)

In [4]:
data1.shape

(1205, 12)

In [5]:
data2.shape

(50, 10)

In [6]:
full_data.shape
del data1, data2

데이터 형태 체크

In [7]:
full_data.head

<bound method NDFrame.head of             일자 요일  본사정원수  본사휴가자수  본사출장자수  본사시간외근무명령서승인건수  현본사소속재택근무자수  \
0   2016-02-01  월   2601      50     150             238          0.0   
1   2016-02-02  화   2601      50     173             319          0.0   
2   2016-02-03  수   2601      56     180             111          0.0   
3   2016-02-04  목   2601     104     220             355          0.0   
4   2016-02-05  금   2601     278     181              34          0.0   
..         ... ..    ...     ...     ...             ...          ...   
45  2021-04-05  월   2973     125     174             704        331.0   
46  2021-04-06  화   2973      76     170             636        364.0   
47  2021-04-07  수   2973      96     214               1        334.0   
48  2021-04-08  목   2973     105     238             509        324.0   
49  2021-04-09  금   2973     259     268               0        229.0   

                                                 조식메뉴  \
0   모닝롤/찐빵  우유/두유/주스 계란후라이  호두죽/쌀밥 (

In [8]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1255 entries, 0 to 49
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1255 non-null   object 
 1   요일              1255 non-null   object 
 2   본사정원수           1255 non-null   int64  
 3   본사휴가자수          1255 non-null   int64  
 4   본사출장자수          1255 non-null   int64  
 5   본사시간외근무명령서승인건수  1255 non-null   int64  
 6   현본사소속재택근무자수     1255 non-null   float64
 7   조식메뉴            1255 non-null   object 
 8   중식메뉴            1255 non-null   object 
 9   석식메뉴            1255 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 127.5+ KB


In [9]:
full_data.shape

(1255, 12)

우리가 전처리 하고 싶은 열은 다음과 같다. ->

In [10]:
full_data.조식메뉴

0     모닝롤/찐빵  우유/두유/주스 계란후라이  호두죽/쌀밥 (쌀:국내산) 된장찌개  쥐...
1     모닝롤/단호박샌드  우유/두유/주스 계란후라이  팥죽/쌀밥 (쌀:국내산) 호박젓국찌...
2     모닝롤/베이글  우유/두유/주스 계란후라이  표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...
3     모닝롤/토마토샌드  우유/두유/주스 계란후라이  닭죽/쌀밥 (쌀,닭:국내산) 근대국...
4     모닝롤/와플  우유/두유/주스 계란후라이  쇠고기죽/쌀밥 (쌀:국내산) 재첩국  방...
                            ...                        
45    모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...
46    모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...
47    모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...
48    모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...
49    모닝롤/토마토샌드 우유/주스 계란후라이 채소죽/흑미밥 대구지리 애호박나물볶음 양상추...
Name: 조식메뉴, Length: 1255, dtype: object

In [11]:
full_data.중식메뉴

0     쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개  쇠불고기 (쇠고기:호주산) 계란찜 ...
1     쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개  가자미튀김  모둠소세지구이  마늘쫑무...
2     카레덮밥 (쌀,현미흑미:국내산) 팽이장국  치킨핑거 (닭고기:국내산) 쫄면야채무침 ...
3     쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국  주꾸미볶음  부추전  시금치나물  ...
4     쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국  돈육씨앗강정 (돼지고기:국내산) 우엉잡채...
                            ...                        
45    쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...
46    쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...
47    쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...
48    쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...
49    쌀밥/흑미밥/찰현미밥 사골우거지국 해물누룽지탕 청포묵*양념간장 비름나물고추장무침 석...
Name: 중식메뉴, Length: 1255, dtype: object

In [12]:
full_data.석식메뉴

0     쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장  자반고등어구이  두부조림  건파래무침 ...
1     콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국  유산슬 (쇠고기:호주산) 아삭고추무...
2     쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개  황태양념구이 (황태:러시아산) 고기...
3     미니김밥*겨자장 (쌀,현미흑미:국내산) 우동  멕시칸샐러드  군고구마  무피클  포...
4     쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...
                            ...                        
45                      흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기 
46             추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치 
47             단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치 
48                  흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음 
49               흑미밥 맑은버섯국 매운사태조림 춘권*타르타르D 열무나물무침 포기김치 
Name: 석식메뉴, Length: 1255, dtype: object

우리는 이 데이터를 훈련에 사용할게 아니라 단순히 밥을 뭐 먹었는지 명부를 얻고자 한다.
공백 두번과 / 문자열로 구분자를 사용하고 있으며 (), *과 같이 묶여있는 경우 구분자가 아니라 연결하는 의미에서 중요하다고 판단하였음.
따라서 조식, 중식, 석식메뉴에 고유하게 등장한 메뉴값을 구한다.

In [13]:
temp = []
for i in full_data.조식메뉴:
    line = i.split(' ')
    temp.extend(line)

for i in full_data.중식메뉴:
    line = i.split(' ')
    temp.extend(line)

for i in full_data.석식메뉴:
    line = i.split(' ')
    temp.extend(line)


띄어쓰기 두번을 기준으로 구분을 한 데이터는 다음과 같이 분류되었음.

In [14]:
temp

['모닝롤/찐빵',
 '',
 '우유/두유/주스',
 '계란후라이',
 '',
 '호두죽/쌀밥',
 '(쌀:국내산)',
 '된장찌개',
 '',
 '쥐어채무침',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤/단호박샌드',
 '',
 '우유/두유/주스',
 '계란후라이',
 '',
 '팥죽/쌀밥',
 '(쌀:국내산)',
 '호박젓국찌개',
 '',
 '시래기조림',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤/베이글',
 '',
 '우유/두유/주스',
 '계란후라이',
 '',
 '표고버섯죽/쌀밥',
 '(쌀:국내산)',
 '콩나물국',
 '',
 '느타리호박볶음',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤/토마토샌드',
 '',
 '우유/두유/주스',
 '계란후라이',
 '',
 '닭죽/쌀밥',
 '(쌀,닭:국내산)',
 '근대국',
 '',
 '멸치볶음',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤/와플',
 '',
 '우유/두유/주스',
 '계란후라이',
 '',
 '쇠고기죽/쌀밥',
 '(쌀:국내산)',
 '재첩국',
 '',
 '방풍나물',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '팬케익/찐빵',
 '',
 '우유/두유/주스',
 '',
 '계란후라이',
 '',
 '견과류죽/쌀밥',
 '(쌀:국내산)',
 '감자찌개',
 '',
 '명엽채무침',
 '포기김치',
 '(김치:국내산)',
 '',
 '모닝롤/야채샌드',
 '',
 '우유/두유/주스',
 '',
 '계란후라이',
 '',
 '고구마죽/쌀밥',
 '(쌀:국내산)',
 '봄동된장국',
 '',
 '숙주나물',
 '포기김치',
 '(김치:국내산)',
 '',
 '모닝롤/치즈프레즐',
 '',
 '우유/두유/주스',
 '',
 '계란후라이',
 '',
 '잣죽/쌀밥',
 '(쌀:국내산)',
 '민물새우찌개',
 '',
 '콩조림',
 '

/ 문자열을 기준으로 더 나눠준다.

In [15]:
cat = []
for i in temp:
    line = i.split('/')
    cat.extend(line)


처리한 형태는 더욱 깔끔해졌다.

In [16]:
cat

['모닝롤',
 '찐빵',
 '',
 '우유',
 '두유',
 '주스',
 '계란후라이',
 '',
 '호두죽',
 '쌀밥',
 '(쌀:국내산)',
 '된장찌개',
 '',
 '쥐어채무침',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤',
 '단호박샌드',
 '',
 '우유',
 '두유',
 '주스',
 '계란후라이',
 '',
 '팥죽',
 '쌀밥',
 '(쌀:국내산)',
 '호박젓국찌개',
 '',
 '시래기조림',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤',
 '베이글',
 '',
 '우유',
 '두유',
 '주스',
 '계란후라이',
 '',
 '표고버섯죽',
 '쌀밥',
 '(쌀:국내산)',
 '콩나물국',
 '',
 '느타리호박볶음',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤',
 '토마토샌드',
 '',
 '우유',
 '두유',
 '주스',
 '계란후라이',
 '',
 '닭죽',
 '쌀밥',
 '(쌀,닭:국내산)',
 '근대국',
 '',
 '멸치볶음',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '모닝롤',
 '와플',
 '',
 '우유',
 '두유',
 '주스',
 '계란후라이',
 '',
 '쇠고기죽',
 '쌀밥',
 '(쌀:국내산)',
 '재첩국',
 '',
 '방풍나물',
 '',
 '포기김치',
 '(배추,고추가루:국내산)',
 '',
 '팬케익',
 '찐빵',
 '',
 '우유',
 '두유',
 '주스',
 '',
 '계란후라이',
 '',
 '견과류죽',
 '쌀밥',
 '(쌀:국내산)',
 '감자찌개',
 '',
 '명엽채무침',
 '포기김치',
 '(김치:국내산)',
 '',
 '모닝롤',
 '야채샌드',
 '',
 '우유',
 '두유',
 '주스',
 '',
 '계란후라이',
 '',
 '고구마죽',
 '쌀밥',
 '(쌀:국내산)',
 '봄동된장국',
 '',
 '숙주나물',
 '포기김치',
 '(김치:

공백값을 가지고 있는 리스트는 무의미한 데이터이다.
이 값을 제거하였다.

In [17]:
while '' in cat:
    cat.remove('')

공백값을 지운 리스트의 형태이다.
이를 중복값을 제거해 데이터 인덱싱 값으로 전환해 주겠다.

In [18]:
catdict = {val : idx for idx, val in enumerate(set(cat))}
catdict

{'(닭:국내산)': 0,
 '요구르트': 1,
 '갈치양념조림': 2,
 '모둠소시지구이': 3,
 '늙은호박죽': 4,
 '청경채찜': 5,
 '맑은연두부탕': 6,
 '과일샐러드': 7,
 '미소장국': 8,
 '식빵피자': 9,
 '찹쌀밥': 10,
 '흑임자연근샐러드': 11,
 '버섯죽': 12,
 '그린샐러드*참깨D': 13,
 '쌈추': 14,
 '단호박크림스프': 15,
 '간장계란장': 16,
 '채소계란찜': 17,
 '부추양파무침': 18,
 '카야잼샌드': 19,
 '계란야채말이': 20,
 '쇠고기버섯국': 21,
 '콩나물밥*달래장': 22,
 '파채상추무침': 23,
 '쌈&야채스틱': 24,
 '(돈민찌:국내산)': 25,
 '키위드레싱샐러드': 26,
 '찹쌀호떡': 27,
 '치킨핑거*요거트소스': 28,
 '제육미나리볶음': 29,
 '참나물겉절이': 30,
 '닭살겨자냉채': 31,
 '데리야끼파닭': 32,
 '(황태:러시아)': 33,
 '부추깻잎전': 34,
 '미역나물': 35,
 '명엽채무침': 36,
 '해파리냉채': 37,
 '취나물된장무침': 38,
 '스팸계란전': 39,
 '고갈비구이': 40,
 '삼겹살마늘볶음밥': 41,
 '장각삼계탕': 42,
 '삼치양념찜': 43,
 '쑥갓쌈&쌈장': 44,
 '냉이국': 45,
 '쌈추전*간장': 46,
 '콩나물국': 47,
 '매실주스': 48,
 '바나나샌드': 49,
 '스프': 50,
 '스프링롤*타르타르D': 51,
 '계란찜': 52,
 '감자만두국': 53,
 '순살깐풍기': 54,
 '부대찌개&쫄면사리': 55,
 '유부채소겨자무침': 56,
 '대패삽겹숙주볶음': 57,
 '순대들깨볶음': 58,
 '해물볶음': 59,
 '매실짱아찌': 60,
 '돈육모듬장조림': 61,
 '청경채사과무침': 62,
 '치커리유자청생채': 63,
 '단배추겉절이': 64,
 '유부주머니국': 65,
 '김실파무침': 66,
 '훈제오리볶음': 67,
 '참치모

사전 제작에 사용한 변수는 지워준다

In [19]:
del full_data, cat, line, temp

각 열에 조식, 점심, 저녁 메뉴에 있어 전처리를 진행한다.
치환을 해야하므로 각 열에 변수를 적용할 필요가 있다.

각 열에 같은 전처리를 하는 함수를 제작한다.

In [20]:
def split_menu(x):
    temp = x.split(' ')
    cat = []
    for j in temp:
        line = j.split('/')
        cat.extend(line)
    while '' in cat:
        cat.remove('')
    return cat

다시 train과 test 데이터셋을 불러온다.

In [21]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train과 test셋의 메뉴를 리스트화 시켜준다.

In [22]:
train_data['조식메뉴'] = train_data['조식메뉴'].transform(split_menu)
train_data['중식메뉴'] = train_data['중식메뉴'].transform(split_menu)
train_data['석식메뉴'] = train_data['석식메뉴'].transform(split_menu)
test_data['조식메뉴'] = test_data['조식메뉴'].transform(split_menu)
test_data['중식메뉴'] = test_data['중식메뉴'].transform(split_menu)
test_data['석식메뉴'] = test_data['석식메뉴'].transform(split_menu)

아, 굳이 라벨인코딩 할 필요가 없었구나
이제 이 식사 데이터를 다차원으로 펼처준다.
메뉴는 16개를 초과하지 않으므로 연습삼아 50개 벡터로 임베딩 해 보겠다.
마찬가지로, 조식, 중식, 석식이므로 이 작업은 세번에 나누어서 걸처 진행하였다.

-> 원래라면 데이터셋의 식사가 없는 값에대해선 word2vec을 하면 안되는데 우리는 이 값 예측이 아니라 임베딩 목적이므로, 식사가 없는 "-" 하나도 매핑하는 식으로 진행한다.

In [23]:
# 콜백을 설정한다.
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

class Callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
        self.training_loss = []

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 1:
            current_loss = loss
        else:
            current_loss = loss - self.loss_previous_step
        print(f"Loss after epoch {self.epoch}: {current_loss}")
        self.training_loss.append(current_loss)
        self.epoch += 1
        self.loss_previous_step = loss

고유한 메뉴의 개수는 3282개. 256차원에 임베딩한다.
임베딩 결과로 나오는 출력 벡터는 임의의 50차원에 할당시키는 식으로 코딩을 진행하였다.

In [24]:
len(catdict)

3282

같이 임베딩을 하는게 옳다고도 생각했지만, 아침 점심 저녁마다 의미가 다르다고 생각하여 따로 임베딩을 실시한다.

In [25]:
model_1 = Word2Vec(window = 10, min_count = 1, sg = 0 ,negative = 20, workers = multiprocessing.cpu_count()-1, vector_size =50)
print(model_1)

logging.disable(logging.NOTSET) # enable logging
t = time()

X = list(train_data['조식메뉴'])
model_1.build_vocab(X)

logging.disable(logging.INFO) # disable logging
callback = Callback() # instead, print out loss for each epoch
t = time()

model_1.train(X, total_examples = model_1.corpus_count, epochs = 100, compute_loss = True, callbacks = [callback])

model_1.save("조식메뉴.model")

2022-08-06 17:36:32,864 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=50, alpha=0.025)', 'datetime': '2022-08-06T17:36:32.864793', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
2022-08-06 17:36:32,866 : INFO : collecting all words and their counts
2022-08-06 17:36:32,868 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-08-06 17:36:32,874 : INFO : collected 822 word types from a corpus of 14416 raw words and 1205 sentences
2022-08-06 17:36:32,874 : INFO : Creating a fresh vocabulary
2022-08-06 17:36:32,881 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 822 unique words (100.0%% of original 822, drops 0)', 'datetime': '2022-08-06T17:36:32.881747', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'ev

Word2Vec(vocab=0, vector_size=50, alpha=0.025)
Loss after epoch 1: 28710.1796875
Loss after epoch 2: 18451.390625
Loss after epoch 3: 17459.84375
Loss after epoch 4: 16879.2109375
Loss after epoch 5: 16726.6015625
Loss after epoch 6: 16772.78125
Loss after epoch 7: 16494.6484375
Loss after epoch 8: 16444.46875
Loss after epoch 9: 16440.84375
Loss after epoch 10: 16391.796875
Loss after epoch 11: 16209.8125
Loss after epoch 12: 16016.546875
Loss after epoch 13: 16064.4375
Loss after epoch 14: 16316.8125
Loss after epoch 15: 15960.921875
Loss after epoch 16: 16077.171875
Loss after epoch 17: 15613.8125
Loss after epoch 18: 15661.21875
Loss after epoch 19: 15555.1875
Loss after epoch 20: 15462.40625
Loss after epoch 21: 15468.21875
Loss after epoch 22: 14881.21875
Loss after epoch 23: 14991.5625
Loss after epoch 24: 14844.40625
Loss after epoch 25: 14639.375
Loss after epoch 26: 13999.28125
Loss after epoch 27: 14100.78125
Loss after epoch 28: 13963.53125
Loss after epoch 29: 13937.375
Lo

In [26]:
model_1.corpus_count

1205

In [27]:
model_2 = Word2Vec(vector_size =50, window = 10, min_count = 1,sg = 0,negative = 20, workers = multiprocessing.cpu_count()-1)
print(model_2)

logging.disable(logging.NOTSET) # enable logging
t = time()

X = list(train_data['중식메뉴'])
model_2.build_vocab(X)

logging.disable(logging.INFO) # disable logging
callback = Callback() # instead, print out loss for each epoch
t = time()

model_2.train(X, total_examples = model_2.corpus_count, epochs = 100, compute_loss = True, callbacks = [callback])

model_2.save("중식메뉴.model")

2022-08-06 17:36:36,921 : INFO : collecting all words and their counts
2022-08-06 17:36:36,921 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-08-06 17:36:36,926 : INFO : collected 1763 word types from a corpus of 11155 raw words and 1205 sentences
2022-08-06 17:36:36,927 : INFO : Creating a fresh vocabulary
2022-08-06 17:36:36,938 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 1763 unique words (100.0%% of original 1763, drops 0)', 'datetime': '2022-08-06T17:36:36.938900', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'prepare_vocab'}
2022-08-06 17:36:36,938 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 11155 word corpus (100.0%% of original 11155, drops 0)', 'datetime': '2022-08-06T17:36:36.938900', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', '

Word2Vec(vocab=0, vector_size=50, alpha=0.025)


2022-08-06 17:36:37,000 : INFO : estimated required memory for 1763 words and 50 dimensions: 1586700 bytes
2022-08-06 17:36:37,001 : INFO : resetting layer weights
2022-08-06 17:36:37,004 : INFO : Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2022-08-06T17:36:37.004725', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'build_vocab'}


Loss after epoch 1: 62463.328125
Loss after epoch 2: 30916.3359375
Loss after epoch 3: 25995.5078125
Loss after epoch 4: 24694.03125
Loss after epoch 5: 24250.03125
Loss after epoch 6: 23832.578125
Loss after epoch 7: 23921.203125
Loss after epoch 8: 24087.171875
Loss after epoch 9: 23868.75
Loss after epoch 10: 24085.28125
Loss after epoch 11: 23789.28125
Loss after epoch 12: 23652.8125
Loss after epoch 13: 23601.46875
Loss after epoch 14: 23313.46875
Loss after epoch 15: 23125.21875
Loss after epoch 16: 22986.5625
Loss after epoch 17: 22647.75
Loss after epoch 18: 22456.65625
Loss after epoch 19: 22386.0625
Loss after epoch 20: 22158.34375
Loss after epoch 21: 22087.21875
Loss after epoch 22: 22091.3125
Loss after epoch 23: 22050.6875
Loss after epoch 24: 21698.75
Loss after epoch 25: 21455.125
Loss after epoch 26: 21389.875
Loss after epoch 27: 21038.5
Loss after epoch 28: 20954.9375
Loss after epoch 29: 20485.5
Loss after epoch 30: 20593.0625
Loss after epoch 31: 20519.125
Loss aft

In [28]:
model_3 = Word2Vec(vector_size =50, window = 10, min_count = 1,sg = 0,negative = 20, workers = multiprocessing.cpu_count()-1)
print(model_3)

logging.disable(logging.NOTSET) # enable logging
t = time()

X = list(train_data['석식메뉴'])
model_3.build_vocab(X)

logging.disable(logging.INFO) # disable logging
callback = Callback() # instead, print out loss for each epoch
t = time()

model_1.train(X, total_examples = model_3.corpus_count,epochs = 100, compute_loss = True, callbacks = [callback])

model_3.save("석식메뉴.model")

2022-08-06 17:36:41,593 : INFO : collecting all words and their counts
2022-08-06 17:36:41,594 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-08-06 17:36:41,598 : INFO : collected 1668 word types from a corpus of 10143 raw words and 1205 sentences
2022-08-06 17:36:41,599 : INFO : Creating a fresh vocabulary
2022-08-06 17:36:41,610 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 1668 unique words (100.0%% of original 1668, drops 0)', 'datetime': '2022-08-06T17:36:41.610410', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'prepare_vocab'}
2022-08-06 17:36:41,611 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 10143 word corpus (100.0%% of original 10143, drops 0)', 'datetime': '2022-08-06T17:36:41.611407', 'gensim': '4.1.2', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', '

Word2Vec(vocab=0, vector_size=50, alpha=0.025)
Loss after epoch 1: 11161.556640625
Loss after epoch 2: 10165.7890625
Loss after epoch 3: 9624.244140625
Loss after epoch 4: 9627.82421875
Loss after epoch 5: 9236.2734375
Loss after epoch 6: 9052.69140625
Loss after epoch 7: 8734.43359375
Loss after epoch 8: 8917.6015625
Loss after epoch 9: 8821.84375
Loss after epoch 10: 8459.28125
Loss after epoch 11: 8280.734375
Loss after epoch 12: 8400.2265625
Loss after epoch 13: 8465.8359375
Loss after epoch 14: 8332.6015625
Loss after epoch 15: 8536.421875
Loss after epoch 16: 8163.359375
Loss after epoch 17: 8099.71875
Loss after epoch 18: 8132.5
Loss after epoch 19: 7892.828125
Loss after epoch 20: 8059.3125
Loss after epoch 21: 8100.125
Loss after epoch 22: 8395.375
Loss after epoch 23: 8055.75
Loss after epoch 24: 7826.65625
Loss after epoch 25: 7976.28125
Loss after epoch 26: 7732.359375
Loss after epoch 27: 7922.859375
Loss after epoch 28: 7696.484375
Loss after epoch 29: 7774.453125
Loss af

In [29]:
embedding_matrix1 = model_1.wv.get_normed_vectors()

In [30]:
embedding_matrix1

array([[ 0.1343294 ,  0.03993437, -0.01115934, ..., -0.05964847,
        -0.0941304 , -0.04544953],
       [ 0.01929653,  0.25939214, -0.12432472, ..., -0.06837619,
         0.12845968,  0.00826162],
       [-0.00588814,  0.3728333 ,  0.01846328, ..., -0.2557516 ,
        -0.04785429, -0.13774262],
       ...,
       [ 0.19878817,  0.16574757,  0.08287695, ..., -0.13435557,
        -0.03953397, -0.07488822],
       [ 0.18397367,  0.17205966,  0.07364056, ..., -0.13237938,
        -0.02312046, -0.07748805],
       [ 0.0955886 ,  0.30031428,  0.0433103 , ..., -0.14807832,
        -0.03552889, -0.03834133]], dtype=float32)

In [31]:
print(model_1)

Word2Vec(vocab=822, vector_size=50, alpha=0.025)


In [32]:
train_data['조식메뉴'].shape

(1205,)

In [33]:
sum(train_data['조식메뉴'].apply(lambda x: len(x)) <= 1)

0

이제 이렇게 훈련한 모델의 값을 벡터화 해서 풀어서 저장해줘야한다.
train셋에 기준으로 word2vec 작업을 실시하였고
word2vec모델을 입력해 입력한 변수 기준, 데이터 벡터화를 실시하였음.
test셋에 대해서는 이 모델만을 이용해 벡터화를 실시함.


벡터로 푸는 임베딩은 이 글보고 참조함.

https://dylancastillo.co/nlp-snippets-cluster-documents-using-word2vec/

In [34]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

In [35]:
tokenized_docs = list(train_data['조식메뉴'])
vectorized_docs_1 = vectorize(tokenized_docs, model=model_1)
len(vectorized_docs_1), len(vectorized_docs_1[0])

(1205, 50)

In [36]:
vectorized_docs_1

[array([ 3.4506336e-01,  8.4102952e-01, -2.4478072e-01, -9.9739715e-02,
         4.2242160e-01,  9.0011835e-02, -5.3227508e-01, -4.6197096e-01,
         9.6157834e-02,  5.4221320e-01,  7.7058665e-02,  1.6484207e-01,
         7.4495667e-01,  2.7373633e-01,  6.6703252e-02,  3.9188913e-01,
        -8.4435480e-04,  1.8571506e-01, -1.5239979e-01, -2.9064106e-02,
         6.0457122e-01,  1.0219537e-01, -1.5876254e-01, -9.2235625e-01,
         3.2877117e-02, -2.5962248e-01,  2.2998804e-01, -1.9168450e-01,
         1.3938937e-01, -3.7141040e-02, -1.3061063e-01, -4.6312857e-01,
        -3.5647428e-01,  7.6357111e-02, -4.2463633e-01,  1.8119504e-01,
         5.0466627e-01, -2.5609744e-01, -2.7084908e-02, -2.7882355e-01,
        -8.2017505e-01,  3.1777300e-02,  8.9952946e-01,  2.0296635e-01,
        -7.1628600e-02,  8.6906709e-02,  1.2767714e-01, -7.3363936e-01,
        -8.1215821e-02, -1.9419023e-01], dtype=float32),
 array([ 5.3775138e-01,  7.7256542e-01, -2.2772752e-01, -4.0358323e-01,
       

In [37]:
tokenized_docs = list(train_data['중식메뉴'])
vectorized_docs_2 = vectorize(tokenized_docs, model=model_1)
len(vectorized_docs_2), len(vectorized_docs_2[0])

(1205, 50)

In [38]:
tokenized_docs = list(train_data['석식메뉴'])
vectorized_docs_3 = vectorize(tokenized_docs, model=model_1)
len(vectorized_docs_3), len(vectorized_docs_3[0])

(1205, 50)

In [39]:
tokenized_docs = list(test_data['조식메뉴'])
test_vectorized_docs_1 = vectorize(tokenized_docs, model=model_1)
len(test_vectorized_docs_1), len(test_vectorized_docs_1[0])

(50, 50)

In [40]:
tokenized_docs = list(test_data['중식메뉴'])
test_vectorized_docs_2 = vectorize(tokenized_docs, model=model_1)
len(test_vectorized_docs_2), len(test_vectorized_docs_2[0])

(50, 50)

In [41]:
tokenized_docs = list(test_data['석식메뉴'])
test_vectorized_docs_3 = vectorize(tokenized_docs, model=model_1)
len(test_vectorized_docs_3), len(test_vectorized_docs_3[0])

(50, 50)

In [42]:
train1 = pd.DataFrame(vectorized_docs_1, columns = ['조식'+str(i) for i in range(1,51)])
train2 = pd.DataFrame(vectorized_docs_2, columns = ['중식'+str(i) for i in range(1,51)])
train3 = pd.DataFrame(vectorized_docs_3, columns = ['석식'+str(i) for i in range(1,51)])

test1 = pd.DataFrame(test_vectorized_docs_1, columns = ['조식'+str(i) for i in range(1,51)])
test2 = pd.DataFrame(test_vectorized_docs_2, columns = ['중식'+str(i) for i in range(1,51)])
test3 = pd.DataFrame(test_vectorized_docs_3, columns = ['석식'+str(i) for i in range(1,51)])

In [43]:
trains = pd.concat([train1,train2,train3],axis = 1)
trains

,조식1,조식2,조식3,조식4,조식5,조식6,조식7,조식8,조식9,조식10,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,0.345063,0.841030,-0.244781,-0.099740,0.422422,0.090012,-0.532275,-0.461971,0.096158,0.542213,...,-0.677969,0.026655,0.521240,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326
1,0.537751,0.772565,-0.227728,-0.403583,0.405390,0.161530,-0.509615,-0.304381,0.054612,0.605921,...,0.218089,-0.672441,0.787501,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869
2,0.430975,0.564405,0.014996,-0.049187,0.444255,0.224213,-0.644300,-0.167529,-0.004084,0.501536,...,-0.149827,0.259681,0.897299,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068
3,0.695163,0.915111,-0.464399,-0.160095,0.431543,0.087458,-0.182642,-0.082155,-0.413177,0.653328,...,-0.257097,-0.550777,1.180077,0.568659,-0.483393,1.447895,-0.850024,-0.630203,-0.074369,0.335166
4,0.519855,0.525534,-0.039259,-0.389989,0.470291,0.145550,-0.738665,-0.592160,0.076944,0.407282,...,-0.289660,0.011186,1.035358,0.692184,-0.199249,0.299185,-0.974069,-0.377276,-0.117614,-0.063785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.205926,1.241737,0.267033,0.033066,-0.079005,-0.399070,0.129195,0.337736,-0.808858,-0.338935,...,0.157136,-0.048317,0.516620,-0.263557,0.738902,0.129731,0.395461,-0.805688,-0.497606,0.216988
1201,-0.045411,1.020490,0.032205,-0.095292,0.015577,-0.401353,0.250021,0.675270,-0.662287,-0.404712,...,0.681265,-0.339047,0.113666,0.122566,0.351240,-0.328923,-0.620603,-0.696990,0.313707,0.084509
1202,0.246507,1.052156,0.344443,0.081379,-0.018135,-0.439428,-0.019931,0.252910,-0.819690,-0.437098,...,1.046267,-0.648388,0.122432,0.036668,0.654255,-0.277724,-0.561344,-0.290120,-0.088212,-0.819337
1203,0.092769,1.107356,0.117192,0.020893,0.032035,-0.375386,0.119544,0.367761,-0.783446,-0.297816,...,1.090084,0.055053,0.210113,-0.062785,-0.246567,-0.333712,0.029828,-0.845187,-0.185106,-0.561755


In [44]:
tests = pd.concat([test1,test2,test3],axis = 1)
tests

,조식1,조식2,조식3,조식4,조식5,조식6,조식7,조식8,조식9,조식10,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,0.318706,1.173837,0.228873,-0.016794,-0.130306,-0.372083,0.075624,0.260103,-0.790065,-0.497801,...,0.855815,-0.676641,0.008660,0.049113,0.279932,-0.255711,-0.784561,-0.287553,-0.123809,-0.417225
1,0.273322,0.871655,0.077482,0.143496,0.146469,-0.306999,0.024035,0.536411,-0.598712,-0.342533,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.068761,1.216680,0.287047,0.108864,-0.010989,-0.396852,0.235242,0.301477,-0.753358,-0.324507,...,1.274020,-0.963867,0.068798,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112
3,0.051651,1.120480,0.057369,0.071547,-0.020465,-0.314478,0.020112,0.298894,-0.814068,-0.307893,...,1.215914,-0.308417,0.200664,-0.110430,0.222746,-0.128564,-0.394535,-0.758083,-0.181848,-0.035184
4,0.405712,1.120068,0.380671,0.311570,-0.104281,-0.354468,0.221850,0.212914,-0.846844,-0.205104,...,2.021988,-0.430297,-0.483459,-0.619689,0.896096,-0.793497,-0.771993,-0.141780,0.286809,-0.650598
5,0.178310,1.330413,0.157849,0.013180,-0.005796,-0.557800,0.075651,0.255392,-0.876416,-0.357451,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.078337,0.896581,0.389845,0.257165,0.173091,-0.262230,-0.042740,0.275614,-0.664603,-0.177050,...,1.274020,-0.963867,0.068798,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112
7,0.060798,1.234879,0.586699,0.087984,-0.158621,-0.461822,-0.236837,0.047030,-0.752259,-0.517390,...,-0.805625,-0.938924,1.104038,1.250359,-1.304088,1.062914,-0.232308,-1.033581,-0.420452,0.128777
8,0.089839,0.911408,0.390926,-0.014367,0.088655,-0.410811,0.157443,0.487644,-0.688883,-0.478411,...,1.137758,-0.795042,-0.075241,0.048106,0.274380,-0.382176,-0.622488,-0.425739,0.116015,-0.470904
9,0.124511,1.134649,0.235986,-0.142773,-0.149260,-0.411376,0.175528,0.369362,-0.731373,-0.425775,...,1.894431,-0.407195,-0.303128,-1.115092,0.632459,-1.117839,-0.496161,-0.377495,0.645056,-0.565803


이제 조식, 중식, 석식메뉴에 결합이 완료되었으니
원본 테이블에 연결하여 준다.


In [45]:
train_data = pd.concat([train_data, trains], axis = 1)
test_data = pd.concat([test_data, tests], axis = 1)

train_data.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,"[모닝롤, 찐빵, 우유, 두유, 주스, 계란후라이, 호두죽, 쌀밥, (쌀:국내산),...","[쌀밥, 잡곡밥, (쌀,현미흑미:국내산), 오징어찌개, 쇠불고기, (쇠고기:호주산)...","[쌀밥, 잡곡밥, (쌀,현미흑미:국내산), 육개장, 자반고등어구이, 두부조림, 건파...",...,-0.677969,0.026655,0.521240,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326
1,2016-02-02,화,2601,50,173,319,0.0,"[모닝롤, 단호박샌드, 우유, 두유, 주스, 계란후라이, 팥죽, 쌀밥, (쌀:국내산...","[쌀밥, 잡곡밥, (쌀,현미흑미:국내산), 김치찌개, 가자미튀김, 모둠소세지구이, ...","[콩나물밥*양념장, (쌀,현미흑미:국내산), 어묵국, 유산슬, (쇠고기:호주산), ...",...,0.218089,-0.672441,0.787501,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869
2,2016-02-03,수,2601,56,180,111,0.0,"[모닝롤, 베이글, 우유, 두유, 주스, 계란후라이, 표고버섯죽, 쌀밥, (쌀:국내...","[카레덮밥, (쌀,현미흑미:국내산), 팽이장국, 치킨핑거, (닭고기:국내산), 쫄면...","[쌀밥, 잡곡밥, (쌀,현미흑미:국내산), 청국장찌개, 황태양념구이, (황태:러시아...",...,-0.149827,0.259681,0.897299,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068
3,2016-02-04,목,2601,104,220,355,0.0,"[모닝롤, 토마토샌드, 우유, 두유, 주스, 계란후라이, 닭죽, 쌀밥, (쌀,닭:국...","[쌀밥, 잡곡밥, (쌀,현미흑미:국내산), 쇠고기무국, 주꾸미볶음, 부추전, 시금치...","[미니김밥*겨자장, (쌀,현미흑미:국내산), 우동, 멕시칸샐러드, 군고구마, 무피클...",...,-0.257097,-0.550777,1.180077,0.568659,-0.483393,1.447895,-0.850024,-0.630203,-0.074369,0.335166
4,2016-02-05,금,2601,278,181,34,0.0,"[모닝롤, 와플, 우유, 두유, 주스, 계란후라이, 쇠고기죽, 쌀밥, (쌀:국내산)...","[쌀밥, 잡곡밥, (쌀,현미흑미:국내산), 떡국, 돈육씨앗강정, (돼지고기:국내산)...","[쌀밥, 잡곡밥, (쌀,현미흑미:국내산), 차돌박이찌개, (쇠고기:호주산), 닭갈비...",...,-0.289660,0.011186,1.035358,0.692184,-0.199249,0.299185,-0.974069,-0.377276,-0.117614,-0.063785


길이 맞는지 확인.

In [46]:
train_data.shape

(1205, 162)

In [47]:
test_data.shape

(50, 160)

In [51]:
train_data.to_csv("Embedding_train_data.csv", index = False, encoding = 'utf-8-sig')
test_data.to_csv("Embedding_test_data.csv", index = False, encoding = 'utf-8-sig')

뭔가 불안해서, 체크해봤는데 값 존재함.

In [49]:
train_data.중식계

0       1039.0
1        867.0
2       1017.0
3        978.0
4        925.0
         ...  
1200    1093.0
1201     832.0
1202     579.0
1203    1145.0
1204    1015.0
Name: 중식계, Length: 1205, dtype: float64

In [50]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Columns: 162 entries, 일자 to 석식50
dtypes: float32(50), float64(103), int64(4), object(5)
memory usage: 1.3+ MB
